# RF Model Tuning

In this notebook, we will take the folds generated by the OU class in the previous notebook  and try to find the best set of parameters for our Random Forest to perform binary classification. 

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import pickle

import OU


%load_ext autoreload
%autoreload 2

In [2]:
save_dir = './data/'

directly read splits results calculated from "SVM Model Tuning.ipynb"

In [3]:
splits = np.load(save_dir + "/splits.npy", allow_pickle=True)

In [4]:
multi_cv_df = pd.read_csv(save_dir + 'df.csv', index_col = 0)
multi_cv_labels = pd.read_csv(save_dir + 'labels.csv', index_col = 0)

# Gridsearch

We want to find the optimal hyperparameters for our XGB by exploring all combinations of possible hyperparameter

In [9]:
params = [{
            'max_depth': [10, 30],
            'min_samples_leaf': [2, 4],
            'min_samples_split': [2, 5],
            'n_estimators': [20, 50]
        }]

In [10]:
# Use all cores (n_jobs-1)
gridcv = GridSearchCV(RandomForestClassifier(), params, verbose = 1, cv = list(splits), n_jobs = -1, 
                    scoring = ['precision'], refit = False)

gridcv.fit(multi_cv_df, multi_cv_labels)

Fitting 399 folds for each of 16 candidates, totalling 6384 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y wa

GridSearchCV(cv=[array([RangeIndex(start=0, stop=2000, step=1),
       RangeIndex(start=2000, stop=2100, step=1)], dtype=object),
                 array([RangeIndex(start=2100, stop=4100, step=1),
       RangeIndex(start=4100, stop=4200, step=1)], dtype=object),
                 array([RangeIndex(start=4200, stop=6200, step=1),
       RangeIndex(start=6200, stop=6300, step=1)], dtype=object),
                 array([RangeIndex(start=6300, s...
       RangeIndex(start=60800, stop=60900, step=1)], dtype=object),
                 array([RangeIndex(start=60900, stop=62900, step=1),
       RangeIndex(start=62900, stop=63000, step=1)], dtype=object), ...],
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'max_depth': [10, 30], 'min_samples_leaf': [2, 4],
                          'min_samples_split': [2, 5],
                          'n_estimators': [20, 50]}],
             refit=False, scoring=['precision'], verbose=1)

In [11]:
gridcv_results = pd.DataFrame(gridcv.cv_results_)

In [12]:
best_param = gridcv_results[gridcv_results.rank_test_precision == 1]['params'].values[0]
print(f'Best Random Forest parameters after Grid Search is {best_param}.')

Best Random Forest parameters after Grid Search is {'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}.


In [13]:
# save the object to a file
with open(save_dir+'gridsearch_results_rf.pkl', 'wb') as f:
    pickle.dump(gridcv, f)